In [186]:
# MODULES

import numpy as np
import matplotlib.pyplot as plt
from random import random, choice
import typing
from functools import partial
from itertools import tee

In [187]:
# CONSTANTS

FLOATING_DIGITS_IN_DNA = 3  # current: x.xx------------...(quite low precision)
INT_SIZE_IN_DNA = 4  # bits current:2**7 = 128 + 1bit(sign)
FLOAT_IN_DNA_LENGHT = INT_SIZE_IN_DNA / 2 + FLOATING_DIGITS_IN_DNA * 2  # XXXX XX XX
BITS2DNA = {"00": "A", "01": "C", "10": "G", "11": "T"}
DNA2BITS = {"A": "00", "C": "01", "G": "10", "T": "11"}

In [188]:
# CLASSES


class Perceptron:
    def __init__(
        self, weights: list[float|int], bias: float|int, activation: typing.Callable
    ):
        self.weights = np.array(weights)
        self.bias = bias
        self.activation = activation

    def __call__(self, inputs: list[int|float]) -> float:
        npinputs = np.array(inputs)
        return self.activation(sum(npinputs * self.weights)+self.bias)


class Network:
    layers: list[list[Perceptron]]
    activation: typing.Callable
    
    @property
    def shape(self) -> tuple[int, ...]:
        return tuple([len(layer) for layer in self.layers])

    def __call__(self, inputs: list[float|int]):
        for layer in self.layers:
            inputs = [perceptron(inputs) for perceptron in layer]
        return inputs

class DNA:
    def __init__(self, seq: str='') -> None:
        self.seq = seq
    def __str__(self) -> str:
        return self.seq
    def __add__(self, other: 'DNA') -> 'DNA':
        return DNA(self.seq + other.seq)
    def __getitem__(self, key):
        return DNA(self.seq[key])
    def __len__(self):
        return len(self.seq)
    def __iter__(self):
        return iter(self.seq)

In [189]:
# FUNCTIONS


def complete_bin(binary: str, digits: int, isneg: bool | None = None) -> str:
    binary = "0" * (digits - len(binary) - (1 if not isneg == None else 0)) + binary
    return (("1" if isneg else "0") if not isneg == None else "") + binary


def int2bin(n: int) -> str:
    return bin(n).split("b")[1]


def bin2dna(binary: str) -> DNA:
    res = DNA()
    for i in range(0, len(binary), 2):
        word = binary[i : i + 2]
        res += DNA(BITS2DNA[word])
    return res


def float2dna(num: int | float) -> DNA:
    num = round(float(num), FLOATING_DIGITS_IN_DNA)

    integer, fraction = str(num).split(".")
    integer = int(integer)
    fraction = list(fraction)
    if len(fraction) < FLOATING_DIGITS_IN_DNA: fraction += ['0'] * (FLOATING_DIGITS_IN_DNA - len(fraction))
    fraction = map(int, fraction)

    integer_bin = int2bin(integer)
    fraction = map(int2bin, fraction)

    if len(integer_bin) >= INT_SIZE_IN_DNA:
        raise ValueError("too big float value")

    integer_complete = complete_bin(integer_bin, INT_SIZE_IN_DNA, num < 0)
    onedigitcomplete = partial(complete_bin, digits=4)
    fraction = map(onedigitcomplete, fraction)

    fraction = map(bin2dna, fraction)
    return (
        bin2dna(integer_complete) + sum(fraction, DNA())
    )


def dna2float(dna: DNA) -> float:
    integer_dna = dna[:INT_SIZE_IN_DNA]
    fraction = dna[4:]
    fraction = [fraction[i:i+2] for i in range(0, 2*FLOATING_DIGITS_IN_DNA, 2)]
    for f in fraction:
        print(f)

    integer_bin = "".join([DNA2BITS[str(some)] for some in integer_dna])
    first_bin = "".join([DNA2BITS[str(some)] for some in first_dna])
    second_bin = "".join([DNA2BITS[str(some)] for some in second_dna])

    integer = int(integer_bin[1:], 2) * (-1 if int(integer_bin[0]) else 1)
    first = int(first_bin, 2)
    second = int(second_bin, 2)

    return float(f"{integer}.{first}{second}")


def perceptron2dna(perceptron: Perceptron) -> DNA:
    weights_dna = DNA()
    for weight in perceptron.weights:
        weights_dna += float2dna(weight)
    return float2dna(len(perceptron.weights)) + weights_dna + float2dna(perceptron.bias)


def dna2perceptron(dna: DNA, activation: typing.Callable) -> Perceptron:
    inputs = int(dna2float(dna[:8]))
    weights_dna = dna[8 : 8 + (inputs * 8)]
    bias_dna = dna[8 + (inputs * 8) :]

    weights = []
    for i in range(0, (inputs) * 8, 8):
        weight_dna = weights_dna[i : i + 8]
        weights.append(dna2float(weight_dna))

    return Perceptron(np.array(weights), dna2float(bias_dna), activation)


def network2dna(network: Network) -> DNA:
    network_dna = DNA()
    for shape, layer in zip(network.shape, network.layers):
        network_dna += float2dna(shape)
        network_dna += float2dna(8 * 2 + 8 * len(layer[0].weights))
        for perceptron in layer:
            network_dna += perceptron2dna(perceptron)
    return network_dna


def dna2network(dna: DNA) -> Network:
    pass

In [190]:
dna2float(DNA('AACCCAGG'))

CA
GG



NameError: name 'first_dna' is not defined